# Top10 Topic Instagram Crawling

# 게시물, 댓글, 해시태그를 크롤링해봅니다
#   

- 원하든 키워드에 대한 게시물의 URL을 검색하고
- URL을 통해 게시물의 내용, 댓글, 해시태그를 크롤링 합니다
#    

![jpeg](https://i1.wp.com/kokolevel.com/wp-content/uploads/2018/10/Instagram.png?w=1620&ssl=1)
#    

In [2]:
# 키워드에 대한 url을 검색하고 url list에 저장
# 그 url list에 대해 게시물과 댓글을 검색

In [3]:
topics = ['society', 'politics', 'economic', 'foreign', 'culture',
          'entertain', 'sports', 'digital']

total_keyword_ranking10 = [['서울', '출근길', '개혁', '화재', '국정원검찰경찰', '대통령', '한화', '개최', '성추행', '이호진'], ['개혁', '대통령', '518', '한국당', '후보', '망언', '국정원검찰경찰', '이낙연', '이해찬', '전략회의'], ['대비', '전일', '영업익', '홍남기', '작년', '지난해', '상승', '결정', '코스피', '하락한'], ['2019', '트럼프', '미중', '무역협상', '폼페이오', '국가비상사태', '비핵화', '()', '류현진', '--'], ['출근길', '내리는', '내린', '출시', '서울', '김수환', '중부', '곳곳', '추기경', '전국'], ['열혈사제', '이하늬', '공개', '김남길', '이대휘', '졸업', '고아성', '품격', '미소', '황후의'], ['감독', '손흥민', '남북', '치어리더', '단일팀', '2019시즌', '류현진', '가스파리니', '공동', '대한항공'], ['진교영', '네이버', 'KT', '출시', '넥슨', 'AI', '모바일', '개최', '트라하', 'LG유플러스']]

In [4]:
# 해시태그를 검색한 페이지에서 ID(검색에 필요한 url)을 검색하는 함수

def crawl_ID(soup):
    
    ID_list = []
    
    while len(soup) > 16000:
        ID_first_index = soup.find('shortcode')
        ID_last_index = soup.find('edge_media_to_comment')
        ID = soup[ID_first_index+12 : ID_last_index-3]
    
        # url이 아닌것은 pass
        if(len(ID)>20):
            break
    
        # 다음 url을 찾기 위해 슬라이싱
        soup = soup[ID_last_index+100:]
        # list에 추가
        ID_list.append(ID)

    # ID list 중복처리
    ID_list = set(ID_list)
    return ID_list

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
topic_index = 0

topic_list = []
main_content_list = []
comments_list = []


for topic in topics[0:1]:
    
    print('Topic : {}\n'.format(topic))
    
    for keyword in total_keyword_ranking10[topic_index]:
        
        print('{}에 대한 게시물을 검색합니다'.format(keyword))
        
        url = 'https://www.instagram.com/explore/tags/{}'.format(keyword)
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        soup = str(soup)

        ID_list = crawl_ID(soup)
        
        for id in ID_list:
            
            url = 'https://www.instagram.com/p/{}'.format(id)
            driver = webdriver.Chrome('/home/apostcto/ITDA/chromedriver')
            driver.get(url)

            contents_elements = driver.find_element_by_css_selector('.eo2As .gElp9').text
            comments_elements = driver.find_elements(By.CSS_SELECTOR, '.eo2As .gElp9')[1:]

            
            
            for comment_element in comments_elements:
   
                text = comment_element.text
                comment_index = text.find('\n')
    
                comment = text[comment_index+1:]
                comments_list.append(comment)
                
                # DataFrame을 생성하기 위해 추가해줍니다
                topic_list.append(topic)
                main_content_list.append(keyword)
                comments_list.append(comment)
        
            # 차단을 막기위해 sleep
            time.sleep(0.1)
            
    topic_index += 1
    print('\n')